In [12]:
import sys
print(sys.path)

['', '/opt/openrobots/lib/python3.6/site-packages', '/opt/openrobots/lib/python3.6/dist-packages', '/home/tlha/catkin_crocoddyl_ws/install/lib/python3/dist-packages', '/home/tlha/tiago_public_ws/install/lib/python2.7/dist-packages', '/opt/ros/melodic/lib/python2.7/dist-packages', '/usr/lib/python36.zip', '/usr/lib/python3.6', '/usr/lib/python3.6/lib-dynload', '/home/tlha/.local/lib/python3.6/site-packages', '/usr/local/lib/python3.6/dist-packages', '/usr/lib/python3/dist-packages', '/home/tlha/.local/lib/python3.6/site-packages/IPython/extensions', '/home/tlha/.ipython']


In [13]:
import crocoddyl
import pinocchio
import numpy as np
import example_robot_data

robot = example_robot_data.load('tiago')
#robot = example_robot_data.loadTiago(True,'wsg')
robot_model = robot.model
q0 = robot_model.referenceConfigurations["tuck_arm"]
# x0 = np.concatenate([q0, np.zeros(robot_model.nv)])
# robot.q0 = q0
DT = 1e-3
T= 250
target = np.array([0.8, 0.1, 1.1]) 
display = crocoddyl.MeshcatDisplay(robot, 4, 4, False)


You can open the visualizer by visiting the following URL:
http://127.0.0.1:7001/static/


In [3]:
print("referenceConfigurations:", q0)

referenceConfigurations: [ 0.15  0.2  -1.34 -0.2   1.94 -1.57  1.37  0.    0.    0.    0.    0.
  0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.
  0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.
  0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    1.    0.
  1.    0.  ]


In [4]:
print("robot.q0 size:", robot.q0.shape[0])
print(robot.q0)

robot.q0 size: 50
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.
 1. 0.]


In [5]:
robot_model

Nb joints = 49 (nq=50,nv=48)
  Joint 0 universe: parent=0
  Joint 1 torso_lift_joint: parent=0
  Joint 2 arm_1_joint: parent=1
  Joint 3 arm_2_joint: parent=2
  Joint 4 arm_3_joint: parent=3
  Joint 5 arm_4_joint: parent=4
  Joint 6 arm_5_joint: parent=5
  Joint 7 arm_6_joint: parent=6
  Joint 8 arm_7_joint: parent=7
  Joint 9 hand_index_abd_joint: parent=8
  Joint 10 hand_index_virtual_1_joint: parent=9
  Joint 11 hand_index_flex_1_joint: parent=10
  Joint 12 hand_index_virtual_2_joint: parent=11
  Joint 13 hand_index_flex_2_joint: parent=12
  Joint 14 hand_index_virtual_3_joint: parent=13
  Joint 15 hand_index_flex_3_joint: parent=14
  Joint 16 hand_index_joint: parent=8
  Joint 17 hand_little_abd_joint: parent=8
  Joint 18 hand_little_virtual_1_joint: parent=17
  Joint 19 hand_little_flex_1_joint: parent=18
  Joint 20 hand_little_virtual_2_joint: parent=19
  Joint 21 hand_little_flex_2_joint: parent=20
  Joint 22 hand_little_virtual_3_joint: parent=21
  Joint 23 hand_little_flex_3_j

In [6]:
# Create the cost functions
Mref = crocoddyl.FrameTranslation(robot_model.getFrameId("wrist_tool_joint"), target)
state = crocoddyl.StateMultibody(robot.model)
goalTrackingCost = crocoddyl.CostModelFrameTranslation(state, Mref)
xRegCost = crocoddyl.CostModelState(state)
uRegCost = crocoddyl.CostModelControl(state)

In [7]:
# Create cost model per each action model
runningCostModel = crocoddyl.CostModelSum(state)
terminalCostModel = crocoddyl.CostModelSum(state)

# Then let's added the running and terminal cost functions
runningCostModel.addCost("gripperPose", goalTrackingCost, 1e2)
runningCostModel.addCost("stateReg", xRegCost, 1e-4)
runningCostModel.addCost("ctrlReg", uRegCost, 1e-7)
terminalCostModel.addCost("gripperPose", goalTrackingCost, 1e5)
terminalCostModel.addCost("stateReg", xRegCost, 1e-4)
terminalCostModel.addCost("ctrlReg", uRegCost, 1e-7)

In [8]:
# Create the actuation model
actuationModel = crocoddyl.ActuationModelFull(state)

# Create the action model
runningModel = crocoddyl.IntegratedActionModelEuler(
    crocoddyl.DifferentialActionModelFreeFwdDynamics(state, actuationModel, runningCostModel), DT)
terminalModel = crocoddyl.IntegratedActionModelEuler(
    crocoddyl.DifferentialActionModelFreeFwdDynamics(state, actuationModel, terminalCostModel))


# Create the problem
q0 = robot_model.referenceConfigurations["tuck_arm"]
x0 = np.concatenate([q0, pinocchio.utils.zero(state.nv)])
problem = crocoddyl.ShootingProblem(x0, [runningModel] * T, terminalModel)

# Creating the DDP solver for this OC problem, defining a logger
ddp = crocoddyl.SolverDDP(problem)
ddp.setCallbacks([crocoddyl.CallbackVerbose()])

# Solving it with the DDP algorithm
ddp.solve()

# Visualizing the solution in gepetto-viewer
display.displayFromSolver(ddp)

robot_data = robot_model.createData()
xT = ddp.xs[-1]
pinocchio.forwardKinematics(robot_model, robot_data, xT[:state.nq])
pinocchio.updateFramePlacements(robot_model, robot_data)
print('Finally reached = ', robot_data.oMf[robot_model.getFrameId("wrist_tool_joint")].translation.T)

Finally reached =  [0.79880909 0.10023166 1.10034979]


In [9]:
x0.shape[0]

98

In [10]:
display.robot.viewer.jupyter_cell()

In [11]:
xT

array([ 2.73885265e-01,  1.30330087e+00, -5.69163266e+00, -7.33996250e+00,
       -1.14797266e+01, -3.26534341e-01, -8.36487443e-01, -3.87410791e+00,
        1.90527875e+00, -4.05360865e-01, -3.35629828e-01, -2.59636020e-01,
       -1.80488864e-01, -1.17040538e-01, -5.46057689e-02, -4.33838961e-02,
        1.60692228e+00, -1.04150042e-01, -7.94487266e-02, -5.69458347e-02,
       -3.72386176e-02, -2.27412232e-02, -9.36185634e-03,  1.64019316e+00,
       -2.69488307e-01, -2.28262022e-01, -1.74405695e-01, -1.19335673e-01,
       -7.72044843e-02, -3.46152963e-02, -4.33838961e-02,  1.61807953e+00,
       -1.95687157e-01, -1.61183290e-01, -1.21815064e-01, -8.23717236e-02,
       -5.27168294e-02, -2.30645624e-02,  9.80415117e-02, -1.14261347e-02,
        7.01046341e-03,  5.49238271e-03,  2.98208240e-03, -4.33838961e-02,
        6.61376919e-06, -6.31547953e-01,  1.00000000e+00,  0.00000000e+00,
        1.00000000e+00,  0.00000000e+00,  1.44195541e+00,  5.29529851e+01,
       -1.23255263e+02, -